In [ ]:
!git clone https://github.com/AbdullahO/SAMoSSA.git
import numpy as np
dataset = np.load('/content/SAMoSSA/datasets/electricity/electricity.npy', encoding='bytes')

In [ ]:
training_set = dataset[25800:25848]      # Arrays 1 to 25824
validation_set = dataset[25848:25872] # Arrays 25825 to 25872

In [ ]:
import pandas as pd
import numpy as np

number_of_hours, num_users = training_set.shape

# Generate date range
date_range = pd.date_range(start='01/01/2011 00:00', periods=number_of_hours, freq='H')

# Reshape and create pairs of values and user IDs
data = []
for user_id in range(1, num_users + 1):
    for hour, value in enumerate(training_set[:, user_id - 1]):
        data.append([date_range[hour], value, user_id])

# Create DataFrame
df_train = pd.DataFrame(data, columns=['Date', 'Load', 'UserID'])
df_train = df_train[df_train['UserID'] != 223]
df_train = df_train[df_train['UserID'] != 207]
df_train = df_train[df_train['UserID'] != 186]
df_train = df_train[df_train['UserID'] != 304]

In [ ]:
df_train

In [ ]:
import pandas as pd
import numpy as np

number_of_hours, num_users = validation_set.shape

# Generate date range
date_range = pd.date_range(start='17/01/2011 00:00', periods=number_of_hours, freq='H')

# Reshape and create pairs of values and user IDs
data = []
for user_id in range(1, num_users + 1):
    for hour, value in enumerate(validation_set[:, user_id - 1]):
        data.append([date_range[hour], value, user_id])

# Create DataFrame
df_valid = pd.DataFrame(data, columns=['Date', 'Load', 'UserID'])

In [ ]:
type(dataset)

In [ ]:
sampled_user_ids=[ 58,  53,  84, 274, 164, 365, 340, 225, 281,  48,  42, 298, 334,
        63,   3, 229, 262, 104,  64,  27, 133,  61, 245,   2,  67, 337,
       127, 248, 218, 217, 317, 280, 243,  76, 219, 250, 305,  75, 350,
        49,  95, 224, 162, 367,  73, 161, 238, 324,  29, 154]

In [ ]:
import pandas as pd

# Filter the original DataFrame to include only the sampled user IDs
df_train = df_train[df_train['UserID'].isin(sampled_user_ids)]
df_valid = df_valid[df_valid['UserID'].isin(sampled_user_ids)]

In [ ]:
%cd /content/SAMoSSA

In [ ]:
from samossa import SAMoSSA

In [ ]:
import pandas as pd
import numpy as np

# Sample DataFrame (replace this with your actual DataFrame)
df = df_train

# Convert 'Date' to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Pivot the DataFrame
pivoted_df = df.pivot(index='Date', columns='UserID', values='Load')

# Convert the pivoted DataFrame to a NumPy ndarray
result_array = pivoted_df.to_numpy()


In [ ]:
import pandas as pd
import numpy as np

# Sample DataFrame (replace this with your actual DataFrame)
df = df_valid

# Convert 'Date' to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Pivot the DataFrame
pivoted_df = df.pivot(index='Date', columns='UserID', values='Load')

# Convert the pivoted DataFrame to a NumPy ndarray
result_array_valid = pivoted_df.to_numpy()

In [ ]:
T, N = result_array.shape
T,N

In [ ]:
L = int(np.sqrt(N*T/2))
model = SAMoSSA(N, L, )

In [ ]:
# fit model on training set
model.fit(result_array)
# forecast the next 48 steps - forecast validation set
predictions = model.predict(24)

In [ ]:
def calculate_smape(true_values, predicted_values):
    denominator = np.abs(true_values+predicted_values)+0.000001
    diff = np.abs(predicted_values - true_values) / denominator
    return 100 * np.mean(diff)

smape_list = []
for i in range(predictions.shape[1]):
    true_values = result_array_valid[:, i]
    predicted_values = predictions[:, i]
    smape = calculate_smape(true_values, predicted_values)

    if not np.isnan(smape):
        smape_list.append(smape)
        print(f"SMAPE for user {i}: {smape}")

smape_average = sum(smape_list) / len(smape_list)
print(f"Average SMAPE across all users (validation set): {smape_average}")